In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

%matplotlib inline

In [2]:
df = pd.read_csv('ht.csv')



In [10]:
print([df.conversationid.unique()])

[array(['8:live:amudgal76', '8:live:.cid.fe63248b85b39c2f',
       '8:live:8922232643a2326e', ..., '8:live:.cid.a4c67d0b2e72bac6',
       '8:live:.cid.78533d4cb35406cc', '8:live:.cid.2ce9f8dccf1fc085'],
      dtype=object)]


In [16]:
df['uni']=df['conversationid'].unique.apply(lambda x: int(0) if x=='8:live:amudgal76' else np.nan)

In [11]:
import re
df['conversationid']=df.conversationid.replace(re.escape('8:live:amudgal76'),0, regex=True)
df['conversationid']=df.conversationid.replace(re.escape('8:live:.cid.fe63248b85b39c2f'),1, regex=True)
df['conversationid']=df.conversationid.replace(re.escape('8:live:8922232643a2326e'),2, regex=True)
df['conversationid']=df.conversationid.replace(re.escape('8:live:.cid.a4c67d0b2e72bac6'),3, regex=True)
df['conversationid']=df.conversationid.replace(re.escape('8:live:.cid.78533d4cb35406cc'),4, regex=True)
df['conversationid']=df.conversationid.replace(re.escape('8:live:.cid.2ce9f8dccf1fc085'),5, regex=True)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['content'],df['conversationid'])

In [4]:
X_train.head(4)

130035                                       Always Welcome
958127    For installing libraries do pip install &lt;li...
340813             Thankyou sir my course is now activated.
859858                    It just does not move beyond this
Name: content, dtype: object

In [15]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=2, batch_size = 32)

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()
print(y_predicted)